In [1]:
import tensorflow as tf
import cv2
import numpy as np
import mysql.connector
import bcrypt
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.metrics.pairwise import cosine_similarity

# Load trained model
face_Model = tf.keras.models.load_model('D:/ICT/3rd-Year/term-2/Security/Model/model_faceID.h5')
face_Model.summary()

# Function to preprocess face image for model input
def preprocess_face(face_img):
    face_img = cv2.resize(face_img, (100, 100))  # Resize to match model input
    face_img = img_to_array(face_img) / 255.0    # Normalize pixel values
    face_img = np.expand_dims(face_img, axis=0)  # Expand dimensions for model
    return face_img

# Connect to MySQL
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="notenote1234",
    database="face_auth_db"
)
cursor = db.cursor()

# Capture face and register user
def register_user():
    cap = cv2.VideoCapture(0)  # Open webcam
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.imshow("Face Capture", frame)
            
            if cv2.waitKey(1) & 0xFF == ord('c'):
                processed_face = preprocess_face(face_img)
                embedding = face_Model.predict(processed_face)[0]
                cap.release()
                cv2.destroyAllWindows()
                
                username = input("Enter username: ")
                password = input("Enter password: ")
                hashed_password = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
                
                cursor.execute("INSERT INTO users (username, password_hash, face_embedding) VALUES (%s, %s, %s)", 
                               (username, hashed_password, embedding.tobytes()))
                db.commit()
                print("✅ User registered successfully!")
                return
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Verify user
def verify_user():
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.imshow("Face Capture", frame)
            
            if cv2.waitKey(1) & 0xFF == ord('c'):
                processed_face = preprocess_face(face_img)
                captured_embedding = face_Model.predict(processed_face)[0]
                cap.release()
                cv2.destroyAllWindows()
                
                username = input("Enter username: ")
                password = input("Enter password: ")
                
                cursor.execute("SELECT password_hash, face_embedding FROM users WHERE username = %s", (username,))
                user = cursor.fetchone()
                
                if user:
                    stored_password, stored_embedding = user
                    stored_embedding = np.frombuffer(stored_embedding, dtype=np.float32)
                    
                    if bcrypt.checkpw(password.encode(), stored_password.encode()):
                        similarity = cosine_similarity([captured_embedding], [stored_embedding])[0][0]
                        if similarity > 0.8:
                            print("✅ Access Granted: Welcome, " + username)
                            return username
                    print("❌ Access Denied: Incorrect credentials or face mismatch")
                else:
                    print("❌ User not found")
                return
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Choose action
choice = input("Do you want to (1) Register or (2) Login? ")
if choice == "1":
    register_user()
elif choice == "2":
    verify_user()
else:
    print("Invalid option.")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 96, 96, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 147456)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    18,874,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,896,470 (72.08 MB)

 Trainable params: 18,896,468 (72.08 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
✅ Access Granted: Welcome, notekub


In [2]:
%pip install bcrypt


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
